In [ ]:
class set_params(tornado.web.RequestHandler):
    def get(self, *args):
        self.post(*args)

    def post(self, *args):

        Ntmp = self.get_arguments('N')
        self.write(" N: ")
        
        for i in range(len(Ntmp)):
            self.write(Ntmp[i])

        tmp = int(self.get_argument('WF'))
        self.write(" WF: ")
        self.write("%d" % tmp) 

In [ ]:
       
class CustomerMap(tornado.web.RequestHandler):
    def get(self, *args):
        self.post(*args)
    
    def post(self, *args):
        
        customerId = int(self.get_argument('IdMusteri'))
        customerIndex = np.where(selectedCustomerIndex2Id==customerId)[0][0]
        
        ax1 = int(self.get_argument('Axis1'))
        ax2 = int(self.get_argument('Axis2'))
        dimensions = np.array([1,1,1,1,0])
        dimensions[ax1] = 0
        dimensions[ax2] = 0

        criteria = int(self.get_argument('Type'))
        if criteria==1:
            plotCriteria = 'sum'
            plotMax = 10
        else:
            plotCriteria = 'binary'
            plotMax = 1
        
        X = loadFundamentalTensorCustomer('files/AllHours_Item_Customer_Tensor.mat', customerIndex, 16)
        #X = collapseTensor(X, [1,0,1,0,0], 'sum')
        X = collapseTensor(X, dimensions, plotCriteria)
        
        plt.figure
        plotTitle = "Sales of Customer %d" % customerId
        plotTensor(X, numPlots=1, title=plotTitle, vmax=plotMax, figsize=(8, 6))
        plt.savefig('./files/%d_%d_%d_%d.png' % (customerId,ax1,ax2,criteria))

        imageUrl = ("45.55.237.86:%s/files/%d_%d_%d_%d.png" % (PORT,customerId,ax1,ax2,criteria))
        
        info = json.dumps({"IdMusteri": customerId, "image_url": imageUrl})
        self.write("%s" % info)
        


In [ ]:
            
class CustomerListForProfile(tornado.web.RequestHandler):
    def get(self, *args):
        self.post(*args)

    def post(self, *args):
        
        productList = self.get_arguments('pId')
        
        numCustomers = 10
        
        percentages = np.random.randint(100, size=(numCustomers)) + 1
        indices = np.argsort(percentages,axis=0)[::-1].flatten()
        percentages = np.sort(percentages,axis=0)[::-1].flatten()

        customerIds = np.random.randint(1000, size=(numCustomers)) + 1
        customerIds = customerIds[indices]

        data = {}
        for i in range(numCustomers):
            data2 = {}
            data2['Percentage'] = int(percentages[i])
            data2['IdMusteri'] = int(customerIds[i])
            data[i+1] = data2 
            
        json_data = json.dumps(data, sort_keys=True)
        self.write(json_data)


In [ ]:
# Given the customer id, this code segment get the sales matrix of the customer, saves it in .png format and displays the json. 
class CustomerSaleJson(tornado.web.RequestHandler):
    def get(self, parameter):
        self.post(parameter)
    
    def post(self, parameter):
        customerId = int(parameter)
        customerIndex = np.where(selectedCustomerIndex2Id==customerId)[0][0]
        
        X = loadFundamentalTensorCustomer('files/AllHours_Item_Customer_Tensor.mat', customerIndex, 16)
        X = collapseTensor(X, [1,0,0,1,0], 'sum')
        SalesTensor = X[0,:,:,0,0] 
        
        plt.figure
        plt.imshow(SalesTensor, aspect='auto', interpolation='nearest',vmin=0, vmax=100)
        plt.savefig('./files/%d.png' % customerId)

        imageUrl = ("45.55.237.86:%s/files/%d.png" % (PORT,customerId))
        
        info = json.dumps({"IdMusteri": customerId, "image_url": imageUrl})
        self.write("%s" % info)

In [ ]:
class CustomerSaleEtailer(tornado.web.RequestHandler):
    def get(self, parameter):
        self.post(parameter)
    
    def post(self, parameter):
        customerId = int(parameter)
        customerIndex = np.where(EtailerSelectedCustomerIndex2Id==customerId)[0][0]
        
        X = loadFundamentalTensorCustomer('files/Etailer_AllHours_Item_Customer_Tensor.mat', customerIndex, 24)
        X = collapseTensor(X, [1,0,0,1,0], 'sum')
        SalesTensor = X[0,:,:,0,0] 
        
        plt.figure
        plt.imshow(SalesTensor, aspect='auto', interpolation='nearest',vmin=0, vmax=100)
        plt.savefig('./files/%d.png' % customerId)
        

        self.write('<html>'
                   '%d <br>' 
                   'Sale Matrix of Customer %d <br>'
                   '<img src=\"/files/%d.png\"><br></body></html>' % (customerIndex,customerId,customerId))
        
        imageUrl = ("45.55.237.86:%s/files/%d.png" % (PORT,customerId))
        
        info = json.dumps({"IdMusteri": customerId, "image_url": imageUrl})
        self.write("%s" % info)
        


In [ ]:
# Given the customer id, this code segment get the sales matrix of the customer, saves it in .png format and displays the image. 
class CustomerSale(tornado.web.RequestHandler):
    def get(self, parameter):
        self.post(parameter)
    
    def post(self, parameter):
        customerId = int(parameter)
        customerIndex = np.where(selectedCustomerIndex2Id==customerId)[0][0]
        
        X = loadFundamentalTensorCustomer('files/AllHours_Item_Customer_Tensor.mat', customerIndex, 16)
        X = collapseTensor(X, [1,0,0,1,0], 'sum')
        SalesTensor = X[0,:,:,0,0] 
        
        plt.figure
        plt.imshow(SalesTensor, aspect='auto', interpolation='nearest',vmin=0, vmax=100)
        plt.savefig('./files/%d.png' % customerId)
        

        self.write('<html>'
                   'Sale Matrix of Customer %d <br>'
                   '<img src=\"/files/%d.png\"><br></body></html>' % (customerId,customerId))
        
        imageUrl = ("45.55.237.86:%s/files/%d.png" % (PORT,customerId))
        
        info = json.dumps({"IdMusteri": customerId, "image_url": imageUrl})
        self.write("%s" % info)
        


In [ ]:
            
# Code segment that will return customer demographic information as a JSON object. 
# The customer id is given. 
# From the api provided, the code segment executes query to get information and convert it to JSON format.
class CustomerInfo(tornado.web.RequestHandler):
    def get(self, parameter):
        self.post(parameter)
    
    def post(self, parameter):
        customerId = int(parameter)
        #self.write("Customer Id: %d " % customerId)
        
        r = requests.get('http://212.57.2.68:93/api/database/musteri?$filter=IdMusteri+eq+%d'%customerId)

        info = json.dumps(r.json())

        #self.write("Customer Demographic Information: %s" % info)
        self.write("%s" % info)
   




In [ ]:
class MainPage2(tornado.web.RequestHandler):
    def get(self):
        self.post()
        
    def post(self):
        self.write('<html><head><h1> Obase Tornado Server </h1></head><br>'
                   '<body><h2> Example Usages </h2><br>'
                   'Last Update: 11.05.2016 <br><br>'
                   '<b> 45.55.237.86:8880/customersOfProfile?jsonData={"ClassId": 0, "ClassName": "Keyifciler", "Products": [{"id": 16540, "name": "ANTRIKOT"}, {"id": 5293, "name": "KAVURMA"}]} </b> <br>'
                   '<b> 45.55.237.86:8880/customerList?pId=7&pId=100&pId=20 </b> <br>'
                   'Given multiple product ids (i.e. Customer Profile), returns 10 users who are suitable for this profile in JSON Format. <br>'
                   'JSON object has multilayer structure such as { "1":{ "Percentage": 100, "IdMusteri": 10271}, "2":{"Percentage": 90, "IdMusteri": 9327} } <br>'
                   'The product ids and customer ids are generated synthetically.' 
                   'This functionality will be updated in the following days for Obase Data. <br><br>'
                   '<b> 45.55.237.86:8880/customerMap?IdMusteri=99737274&Axis1=1&Axis2=3&Type=2 </b> <br>'
                   'Gives the image url of the sales matrix of customer with specified features. <br>'
                   'Axis1 and Axis2 attributes represent the x and y axes of heatmap. The order is not important. <br>'
                   'Axis1 and Axis2 can take values 0(Week), 1(Day of Week), 2(Hour) and 3(Item). <br>'
                   'Type attribute can take the values 1 or 2. <br>'
                   '1 represents the amount of money spent by the customer. ' 
                   '2 is the binary representation of first type (i.e whether the customer buys an item on that day or not). <br>' 
                   'Some customer ids to use for customerMap: 99888001, 991921217, 99958429 <br><br>'
                   '<b> 45.55.237.86:8880/customerInfo/110236 </b> <br>'
                   'Displays demographic information of customer with id = 110236 <br>'
                   'Some customer ids to use for customerInfo: 110236, 100240 <br><br>'
                   '<b> 45.55.237.86:8880/customerSale/99888001 </b> <br>'
                   'Displays heatmap of the sales of customer with id = 99888001 <br>'
                   'Some customer ids to use for customerSale: 99888001, 991921217, 99958429 <br><br>'
                   '<b> 45.55.237.86:8880/customerSaleJson/99888001 </b> <br>'
                   'Displays the image url of the sales matrix of customer with id = 99888001 in JSON format <br>'
                   'Some customer ids to use for customerSaleJson: 99888001, 991921217, 99958429 <br><br>'
                   '</body></html>')
   
